In [1]:
from sklearn.model_selection import train_test_split
import os
import glob
import cv2
import numpy as np # linear algebra
from skimage.io import imread
from skimage.io import imsave
from imageio import imwrite
import matplotlib.pyplot as plt
from skimage.segmentation import mark_boundaries
from skimage.util import montage
from skimage import img_as_ubyte
montage_rgb = lambda x: np.stack([montage(x[:, :, :, i]) for i in range(x.shape[3])], -1)
data_dir = '/media/diego/juan_escutia/MaskRCNN_datset/opencv_projects/SegmentObjectTrain/datasets/objects2019/'
train_image_origin_dir = os.path.join(data_dir, 'train_original')
train_image_dir = os.path.join(data_dir, 'train')
val_image_dir = os.path.join(data_dir, 'val')
backgrounds_dir = os.path.join(data_dir, 'backgrounds')
import gc; gc.enable() # memory is tight
from skimage.morphology import label

In [2]:
BATCH_SIZE = 25
EDGE_CROP = 2
NB_EPOCHS = 20
GAUSSIAN_NOISE = 0.2
UPSAMPLE_MODE = 'DECONV'
# downsampling inside the network
NET_SCALING = None
# downsampling in preprocessing
#IMG_SCALING = (1, 1)
IMG_SCALING = None
# number of validation images to use
VALID_IMG_COUNT = 400
# maximum number of steps_per_epoch in training
MAX_TRAIN_STEPS = 200
AUGMENT_BRIGHTNESS = False

In [3]:
def get_all_imgs():
    img_path = os.path.join(train_image_origin_dir,'annotations')
    images = glob.glob(os.path.join(img_path,'1*.*'))
    return [image.split('/')[-1] for image in images]

In [4]:
# print(get_all_imgs())
TRAIN_IMGS, TEST_IMGS = train_test_split(get_all_imgs())
print(len(TRAIN_IMGS))
print(len(TEST_IMGS))

10125
3375


In [5]:
import random
def cv2_brightness_augment(img):
    hsv = cv2.cvtColor(img,cv2.COLOR_RGB2HSV)
    v = hsv[:,:,2]
    seed = random.uniform(0.5,1.5)
    v = (( v/255.0 ) * seed)*255.0
    hsv[:,:,2] = np.array(np.clip(v,0,255),dtype=np.uint8)
    rgb_final = cv2.cvtColor(hsv,cv2.COLOR_HSV2RGB)
    return rgb_final

In [6]:
def make_image_gen(img_file_list=TRAIN_IMGS, batch_size = BATCH_SIZE):
    all_batches = TRAIN_IMGS
    out_rgb = []
    out_mask = []
    out_classes = []
    img_path = os.path.join(train_image_origin_dir,'images')
    mask_path = os.path.join(train_image_origin_dir,'annotations')
    #Not repeat the batches
    #while True:
    #I Think tha this is slow, comment and generate the random shuffle in main init call
    #np.random.shuffle(all_batches)
    for c_img_id in all_batches:
        c_img_name_split = c_img_id.split('_')
        c_img_class = c_img_id.split('-')[1]
        c_img_name = c_img_id.split('-')[0] + "."  + c_img_id.split('-')[2].split('.')[1]
        c_img = imread(os.path.join(img_path,c_img_name))
        c_img = cv2_brightness_augment(c_img)
        c_mask = imread(os.path.join(mask_path,c_img_id))
        if IMG_SCALING is not None:
            c_img = cv2.resize(c_img,(256,256),interpolation = cv2.INTER_AREA)
            c_mask = cv2.resize(c_mask,(256,256),interpolation = cv2.INTER_AREA)
        c_mask = np.reshape(c_mask,(c_mask.shape[0],c_mask.shape[1],-1))
        c_mask = c_mask > 0
        out_rgb += [c_img]
        out_mask += [c_mask]
        out_classes += [c_img_class]
        if len(out_mask)>=batch_size:
            yield np.stack(out_rgb, 0)/255.0, np.stack(out_mask, 0), out_classes
            out_rgb, out_mask, out_classes=[], [], []

In [8]:
#np.random.shuffle(TRAIN_IMGS)
train_gen = make_image_gen()

In [7]:
train_x, train_y, img_class = next(train_gen)
print('x', train_x.shape, train_x.min(), train_x.max())
print('y', train_y.shape, train_y.min(), train_y.max())
print(img_class)
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize = (30, 10))
batch_rgb = montage_rgb(train_x)
batch_seg = montage(train_y[:, :, :, 0])
ax1.imshow(batch_rgb)
ax1.set_title('Images')
ax2.imshow(batch_seg, cmap='gray')
ax2.set_title('Segmentations')
ax3.imshow(mark_boundaries(batch_rgb, 
                           batch_seg.astype(int)))
ax3.set_title('Outlined Smokes')
fig.savefig('overview.png')

NameError: name 'train_gen' is not defined

In [ ]:
#valid_x, valid_y, valid_class = next(make_image_gen(TEST_IMGS,len(TEST_IMGS)))
#print(valid_x.shape, valid_y.shape, valid_class)
val_gen = make_image_gen(TEST_IMGS, BATCH_SIZE)

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
dg_args = dict(featurewise_center = False, 
                  samplewise_center = False,
                  rotation_range = 30, 
                  width_shift_range = 0.05, 
                  height_shift_range = 0.05, 
                  shear_range = 0.01,
                  zoom_range = [0.7, 1.75],  
                  horizontal_flip = False, 
                  vertical_flip = False,
                  fill_mode = 'constant',
                  data_format = 'channels_last',
                  validation_split=0.2)
# brightness can be problematic since it seems to change the labels differently from the images 
if AUGMENT_BRIGHTNESS:
    dg_args[' brightness_range'] = [0.5, 1.5]
image_gen = ImageDataGenerator(**dg_args)

if AUGMENT_BRIGHTNESS:
    dg_args.pop('brightness_range')
label_gen = ImageDataGenerator(**dg_args)

def create_aug_gen(in_gen, seed = None):
    np.random.seed(seed if seed is not None else np.random.choice(range(9999)))
    for in_x, in_y, in_class in in_gen:
        seed = np.random.choice(range(9999))
        #print seed
        # keep the seeds syncronized otherwise the augmentation to the images is different from the masks
        g_x = image_gen.flow(255*in_x, 
                             batch_size = in_x.shape[0], 
                             seed = seed, 
                             shuffle=False)
        g_y = label_gen.flow(in_y, 
                             batch_size = in_x.shape[0], 
                             seed = seed, 
                             shuffle=False)

        yield next(g_x)/255.0, next(g_y), in_class

In [ ]:
images_backgrounds = glob.glob(os.path.join(backgrounds_dir,'*.*'))
background_images = []
for image_background in images_backgrounds:
    background_image = imread(image_background)
    background_images += [background_image]
#print (random.randint(0,len(images_backgrounds)))
#print (len(background_images))

In [ ]:
def getNextName(image_dir,name_dir):
    img_path = os.path.join(image_dir, name_dir)
    images = glob.glob(os.path.join(img_path,'2*.*'))
    if len(images) > 0:
        images = [os.path.splitext(image.split('/')[-1])[0] for image in images]
        return int(max(images))
    return 2000000

In [ ]:
cur_gen = create_aug_gen(train_gen)
count = 0

t_x = []
t_y = []
in_class = []
initCount = getNextName(train_image_dir, 'images')

img_path = os.path.join(train_image_dir,'images')
mask_path = os.path.join(train_image_dir,'annotations')

while True:
    try:
        t_x, t_y, in_classes = next(cur_gen)
        for index in range(len(t_x)):
            image = img_as_ubyte(t_x[index][...,::-1])
            maskImg = t_y[:, :, :, 0][index]   
            maskImg = np.reshape(maskImg,(maskImg.shape[0],maskImg.shape[1],-1))
            maskImg = maskImg > 0
            mask = img_as_ubyte(maskImg)
            mask_inv = cv2.bitwise_not(mask)
            img1_bg = background_images[random.randint(0,len(images_backgrounds) - 1)][...,::-1]
            img1_bg = cv2.resize(img1_bg, (image.shape[1], image.shape[0]), interpolation = cv2.INTER_AREA)
            img1_bg = cv2.bitwise_and(img1_bg, img1_bg, mask = mask_inv)
            img1_fg = cv2.bitwise_and(image, image, mask = mask)
            scene_image = cv2.add(img1_fg, img1_bg)
            cv2.imwrite(os.path.join(img_path, str(initCount) + ".png"), scene_image)
            imwrite(os.path.join(mask_path, str(initCount) + "-" + in_classes[index] + "-0.png"),t_y[:, :, :, 0][index])
            initCount += 1
        gc.collect()
    except StopIteration:
        break

In [ ]:
gc.collect()

In [ ]:
cur_gen = create_aug_gen(val_gen)
count = 0

v_x = []
v_y = []
in_class = []

initCount = 2000000

img_path = os.path.join(val_image_dir,'images')
mask_path = os.path.join(val_image_dir,'annotations')

while True:
    try:
        v_x, v_y, in_classes = next(cur_gen)
        for index in range(len(v_x)):
            image = img_as_ubyte(v_x[index][...,::-1])
            maskImg = v_y[:, :, :, 0][index]   
            maskImg = np.reshape(maskImg,(maskImg.shape[0],maskImg.shape[1],-1))
            maskImg = maskImg > 0
            mask = img_as_ubyte(maskImg)     
            mask_inv = cv2.bitwise_not(mask)
            img1_bg = background_images[random.randint(0,len(images_backgrounds) - 1)][...,::-1]
            img1_bg = cv2.resize(img1_bg, (image.shape[1], image.shape[0]), interpolation = cv2.INTER_AREA)
            img1_bg = cv2.bitwise_and(img1_bg, img1_bg, mask = mask_inv)
            img1_fg = cv2.bitwise_and(image, image, mask = mask)
            scene_image = cv2.add(img1_fg, img1_bg)
            cv2.imwrite(os.path.join(img_path, str(initCount) + ".png"), scene_image)
            imwrite(os.path.join(mask_path, str(initCount) + "-" + in_classes[index] + "-0.png"),v_y[:, :, :, 0][index])
            initCount += 1
        gc.collect()
    except StopIteration:
        break

In [36]:
print(1)

1
